<h1> Neural Net </h1>

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import subprocess
import time
import os
import glob
from torch.utils.data import Dataset, DataLoader
from lucam import Lucam

# --- 기본 설정 (이전과 동일) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
N = 600
total_cam_roi = (450, -500, 450, -500) # Top Bottom Left Right
first_order_cam_roi = (500, -450, 450, -500) # Top Bottom Left Right

LEARNING_RATE_MODEL = 1e-3
LEARNING_RATE_PHASE = 1e-2

# --- 🧠 중간 깊이의 뉴럴 네트워크 모델 정의 ---
class MediumUNetPropagation(nn.Module):
    def __init__(self, in_channels=2, out_channels=1):
        super(MediumUNetPropagation, self).__init__()

        # --- 인코더 (Contracting Path) ---
        # Level 1
        self.enc1 = self.conv_block(in_channels, 64)
        # Level 2
        self.enc2 = self.conv_block(64, 128)
        # Level 3 (추가된 깊이)
        self.enc3 = self.conv_block(128, 256)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # --- 병목 구간 (Bottleneck) ---
        self.bottleneck = self.conv_block(256, 512)

        # --- 디코더 (Expanding Path) ---
        # Level 3
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(256 + 256, 256) # Skip connection 포함
        # Level 2
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(128 + 128, 128) # Skip connection 포함
        # Level 1
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = self.conv_block(64 + 64, 64)   # Skip connection 포함

        # --- 최종 출력 레이어 ---
        self.out_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def conv_block(self, in_c, out_c):
        """두 개의 3x3 Conv와 ReLU, BatchNorm으로 구성된 기본 블록"""
        return nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            nn.ReLU(inplace=True)
        )

    def forward(self, phase_map):
        # ‼️ 입력 변환: φ -> [cos(φ), sin(φ)]
        if phase_map.dim() == 3: # (B, H, W) -> (B, 1, H, W)
            phase_map = phase_map.unsqueeze(1)

        x_cos = torch.cos(phase_map)
        x_sin = torch.sin(phase_map)
        x = torch.cat([x_cos, x_sin], dim=1) # (B, 2, N, N)

        # --- 인코더 ---
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))

        # --- 병목 ---
        b = self.bottleneck(self.pool(e3))

        # --- 디코더 + Skip Connections ---
        d3 = self.upconv3(b)
        d3 = torch.cat([d3, e3], dim=1)
        d3 = self.dec3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat([d2, e2], dim=1)
        d2 = self.dec2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat([d1, e1], dim=1)
        d1 = self.dec1(d1)

        # --- 출력 ---
        out = self.out_conv(d1)
        return out.squeeze(1) # (B, N, N)

# --- 헬퍼 함수 정의 (이전과 동일) ---
def save_phase_as_image(phase_tensor, filename):
    phase_normalized = (phase_tensor.detach() + torch.pi) / (2 * torch.pi)
    phase_uint8 = (phase_normalized * 255).byte().cpu().numpy()
    Image.fromarray(phase_uint8).save(filename)

def load_and_preprocess_image(path, size=(N, N)):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None: raise FileNotFoundError(f"'{path}' 파일을 찾을 수 없습니다.")
    img = cv2.resize(img, dsize=size)
    img_float = img.astype(np.float32) / np.max(img) # 0~1 사이로 정규화
    return torch.from_numpy(img_float).to('cpu')

# --- 🌟 데이터셋 클래스 정의 🌟 ---
class HolographyDataset(Dataset):
    def __init__(self, image_dir):
        # 이미지 파일 경로 리스트 가져오기
        self.image_paths = glob.glob(os.path.join(image_dir, '*.jpg')) + \
                           glob.glob(os.path.join(image_dir, '*.png'))
                           
        lam = 0.532e-6
        dx = 12.5e-6
        z = 100e-3

        lam = torch.tensor(lam).cuda()
        dx = torch.tensor(dx).cuda()
        z = torch.tensor(z).cuda()

        # 각 이미지에 대한 위상 텐서를 저장할 딕셔너리
        self.phase_tensors = {}
        for path in self.image_paths:
            # 초기 위상은 랜덤으로 생성
            phase = (torch.pi * torch.ones(N,N)).requires_grad_(True)
            self.phase_tensors[path] = phase
            
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        path = self.image_paths[idx]
        target_intensity = load_and_preprocess_image(path)
        target_intensity = target_intensity / torch.max(target_intensity).item()
        target_amplitude = torch.sqrt(target_intensity)
        phase_tensor = self.phase_tensors[path]
        return target_amplitude.to('cuda'), phase_tensor.to('cuda'), path # 경로도 함께 반환하여 추적

# --- 변수 및 모델 초기화 ---
model = MediumUNetPropagation().to(device)

# 🌟 데이터셋 및 데이터로더 생성
# 'images' 폴더에 학습용 이미지를 넣어주세요.
dataset = HolographyDataset(image_dir='./circle')
# 미니배치 크기. GPU 메모리에 따라 조절.
BATCH_SIZE = 1
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# s1, s2 스케일 팩터. 이제 이미지마다 필요할 수 있으나, 우선은 공유
s1 = torch.tensor(1.0, device=device, requires_grad=True)
s2 = torch.tensor(1.0, device=device, requires_grad=True)

# ‼️ 옵티마이저 정의. 이제 위상 텐서는 데이터셋 안에 있으므로, 별도로 최적화
optimizer_model = optim.Adam(list(model.parameters()) + [s2], lr=LEARNING_RATE_MODEL)
# 위상 텐서들을 모아서 phase 옵티마이저에 등록
all_phase_params = list(dataset.phase_tensors.values()) + [s1]
optimizer_phase = optim.Adam(all_phase_params, lr=LEARNING_RATE_PHASE)

loss_fn = torch.nn.MSELoss()

print(f"\n--- 초기화 완료 ---")
print(f"총 {len(dataset)}개의 이미지로 데이터셋 구성 완료.")

Using device: cuda

--- 초기화 완료 ---
총 1개의 이미지로 데이터셋 구성 완료.


In [9]:
from pytorch_msssim import ssim, ms_ssim # Multi-Scale SSIM이 더 성능이 좋을 수 있습니다.
import torch.nn.functional as F

NUM_EPOCHS = 100 # 전체 데이터셋을 몇 번 반복할지

for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*20} Epoch {epoch + 1}/{NUM_EPOCHS} {'='*20}")
    
        # data_loader에서 미니배치 단위로 데이터를 가져옴
    for i, (target_amplitudes, phase_tensors, image_paths) in enumerate(data_loader):
        
        # --- 단계 1: 위상 업데이트 -
        model.eval()
        optimizer_phase.zero_grad()
        
        # U-Net 모델은 배치 입력을 처리할 수 있도록 수정됨
        prediction_for_phase = model(phase_tensors)

        loss_phase = loss_fn(s1 * prediction_for_phase, target_amplitudes**2)
        loss_phase.backward()
        optimizer_phase.step()
        
        print(f"Epoch {epoch+1}, Batch {i+1} [1/2] 위상 업데이트 완료.")

        # --- 단계 2: 모델 업데이트 ---
        # 이 단계에서는 미니배치의 각 이미지에 대해 물리적 실험을 반복해야 함
        
        captured_amplitudes_batch = []
        # 배치 내 각 샘플에 대해 SLM 띄우고 촬영
        for j in range(len(image_paths)):
            phase_to_display = phase_tensors[j]
            
            save_phase_as_image(phase_to_display, 'test.png')
            
            slm_process = subprocess.Popen(['python', 'test.py'])
            time.sleep(2)
            
            # ‼️ 실제 카메라 촬영 로직
            camera = Lucam()
            capture = camera.TakeSnapshot()
            # capture = capture[500:-450, 450:-500]
            capture = cv2.resize(capture, dsize=(N, N))
            cv2.imwrite('captured_image.png', capture)
            
            slm_process.terminate()
            slm_process.wait()

            captured_intensity = load_and_preprocess_image('captured_image.png')
            captured_intensity = captured_intensity / torch.max(captured_intensity).item()
            captured_amp = torch.sqrt(captured_intensity)
            captured_amplitudes_batch.append(captured_amp)
        
        # 촬영된 이미지들을 하나의 배치 텐서로 결합
        captured_amplitudes = torch.stack(captured_amplitudes_batch)
        
        # 모델 학습
        model.train()
        optimizer_model.zero_grad()
        
        # phase_tensors는 업데이트되었지만, 모델 학습에는 이전 상태를 사용해야 함
        prediction_for_model = model(phase_tensors.detach())

        # loss_model = loss_fn(s2 * prediction_for_model, (captured_amplitudes**2).cuda())

        loss_ssim = 1 - ssim(prediction_for_model.unsqueeze(0), (captured_amplitudes.unsqueeze(0)**2).cuda(), data_range=1.0, size_average=True)
        loss_model = loss_ssim

        loss_model.backward()
        optimizer_model.step()
        
        print(f"Epoch {epoch+1}, Batch {i+1} [2/2] 모델 업데이트 완료. Loss: {loss_model.item():.6f}")
        temp = phase_tensors.clone()
        output = model(temp.detach()).detach().cpu().numpy()[0]
        output = (output - np.min(output)) / (np.max(output) - np.min(output))
        output = output * 255
        Image.fromarray(output.astype('uint8')).save('output.png')


Exception ignored in: <function Lucam.__del__ at 0x0000023636CB1E40>
Traceback (most recent call last):
  File "c:\holo\envs\holo\Lib\site-packages\lucam\lucam.py", line 877, in __del__
    assert not self._displaying_window
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\holo\envs\holo\Lib\site-packages\lucam\lucam.py", line 957, in __getattr__
    raise AttributeError(f"'Lucam' object has no attribute '{name}'")
AttributeError: 'Lucam' object has no attribute '_displaying_window'



==================== Epoch 1/100 ====================
Epoch 1, Batch 1 [1/2] 위상 업데이트 완료.
Epoch 1, Batch 1 [2/2] 모델 업데이트 완료. Loss: 1.009946

==================== Epoch 2/100 ====================
Epoch 2, Batch 1 [1/2] 위상 업데이트 완료.
Epoch 2, Batch 1 [2/2] 모델 업데이트 완료. Loss: 0.972435

==================== Epoch 3/100 ====================
Epoch 3, Batch 1 [1/2] 위상 업데이트 완료.
Epoch 3, Batch 1 [2/2] 모델 업데이트 완료. Loss: 0.970831

==================== Epoch 4/100 ====================
Epoch 4, Batch 1 [1/2] 위상 업데이트 완료.
Epoch 4, Batch 1 [2/2] 모델 업데이트 완료. Loss: 0.968198

==================== Epoch 5/100 ====================
Epoch 5, Batch 1 [1/2] 위상 업데이트 완료.
Epoch 5, Batch 1 [2/2] 모델 업데이트 완료. Loss: 0.967725

==================== Epoch 6/100 ====================
Epoch 6, Batch 1 [1/2] 위상 업데이트 완료.
Epoch 6, Batch 1 [2/2] 모델 업데이트 완료. Loss: 0.967306

==================== Epoch 7/100 ====================
Epoch 7, Batch 1 [1/2] 위상 업데이트 완료.
Epoch 7, Batch 1 [2/2] 모델 업데이트 완료. Loss: 0.966722

===================

In [12]:
# ... (이전 코드는 대부분 동일) ...
from pytorch_msssim import ssim

# 손실 함수는 MSE가 더 적합할 수 있습니다. 스페클 패턴의 픽셀 단위 비교이므로.
# loss_fn = torch.nn.MSELoss() 
# 푸리에 도메인 비교에는 SSIM도 효과적일 수 있습니다. 선택적으로 사용해보세요.
loss_fn_fourier = lambda pred, target: 1 - ssim(pred.unsqueeze(1), target.unsqueeze(1), data_range=1.0, size_average=True)
loss_fn_model = torch.nn.MSELoss()

NUM_EPOCHS = 15

for epoch in range(NUM_EPOCHS):
    print(f"\n{'='*20} Epoch {epoch + 1}/{NUM_EPOCHS} {'='*20}")

    for i, (target_amplitudes, phase_tensors, image_paths) in enumerate(data_loader):
        # target_amplitudes는 이제 스페클이 아닌 '최종 목표 이미지'의 진폭입니다.
        # 이 코드에서는 제곱된 intensity를 사용하겠습니다.
        target_image = target_amplitudes**2 

        # --- 단계 1: 위상 업데이트 ---
        model.eval() # 모델은 평가 모드로 두고, 위상만 최적화
        optimizer_phase.zero_grad()

        # 모델을 통해 예상 스페클 패턴을 얻음
        predicted_speckle = model(phase_tensors)

        # ‼️ 핵심: 예상 스페클을 푸리에 변환
        # 푸리에 변환 및 파워 스펙트럼 계산 (intensity of Fourier transform)
        predicted_fft = torch.fft.fftshift(torch.fft.fft2(predicted_speckle), dim=(-2, -1))
        predicted_power_spectrum = torch.abs(predicted_fft)**2
        
        # 정규화 (매우 중요)
        # 배치 내 각 이미지에 대해 독립적으로 정규화
        for b in range(predicted_power_spectrum.shape[0]):
             predicted_power_spectrum[b] /= torch.max(predicted_power_spectrum[b]).item()

        # 푸리에 변환된 결과와 목표 이미지를 비교
        loss_phase = loss_fn_fourier(predicted_power_spectrum, target_image) # 또는 MSE 사용
        loss_phase.backward()
        optimizer_phase.step() # 위상 텐서만 업데이트

        print(f"Epoch {epoch+1}, Batch {i+1} [1/2] 위상 업데이트 완료. Loss: {loss_phase.item():.6f}")

        # --- 단계 2: 모델 업데이트 ---
        # 1단계에서 업데이트된 위상을 실제 시스템에 적용하고 모델을 교정
        
        captured_intensities_batch = []
        for j in range(len(image_paths)):
            phase_to_display = phase_tensors[j]
            save_phase_as_image(phase_to_display, 'test.png') # 디버깅용

            # ... (실제 SLM 구동 및 카메라 촬영 로직) ...
            slm_process = subprocess.Popen(['python', 'test.py'])
            time.sleep(2)
            camera = Lucam()
            capture = camera.TakeSnapshot() 
            capture = cv2.resize(capture, dsize=(N, N))
            cv2.imwrite('captured_image.png', capture)

            slm_process.terminate()
            slm_process.wait()

            # 촬영된 실제 스페클 패턴 로드
            captured_intensity = load_and_preprocess_image('captured_image.png').to(device)
            captured_intensity = captured_intensity / torch.max(captured_intensity).item()
            captured_intensities_batch.append(captured_intensity)

        captured_intensities = torch.stack(captured_intensities_batch)

        # 모델 학습 모드로 전환
        model.train()
        optimizer_model.zero_grad()

        # 모델이 예측한 스페클과 실제 촬영된 스페클을 비교
        # phase_tensors는 위상 업데이트 후의 상태이며, 모델 입력 시 detach()하여
        # 이 손실이 위상에 역전파되지 않도록 함
        prediction_for_model = model(phase_tensors.detach())

        # 스케일 팩터(s2)를 포함하여 손실 계산
        loss_model = loss_fn_model(s2 * prediction_for_model, captured_intensities)
        loss_model.backward()
        optimizer_model.step() # 모델과 s2만 업데이트

        print(f"Epoch {epoch+1}, Batch {i+1} [2/2] 모델 업데이트 완료. Loss: {loss_model.item():.6f}")

        # --- 시각화 및 디버깅 (매우 중요) ---
        # 현재 예측된 푸리에 스펙트럼이 어떻게 생겼는지 확인
        final_recon = predicted_power_spectrum.detach().cpu().numpy()[0]
        Image.fromarray((final_recon * 255).astype('uint8')).save(f'output.png')


==================== Epoch 1/15 ====================
Epoch 1, Batch 1 [1/2] 위상 업데이트 완료. Loss: 0.999463


KeyboardInterrupt: 

In [7]:
speckle = Image.open('captured_image.png').convert('L')
speckle = np.array(speckle)
autocorr = np.fft.fftshift(np.fft.fft2(speckle))
power = np.abs(autocorr)**2
power = power / np.max(power) * 255
Image.fromarray(power.astype('uint8')).save('output.png')